# Forecasting Air Quality with Amazon SageMaker and DeepAR
In this example, we are going to build an air quality forecasting application using Amazon SageMaker and the DeepAR algorithm. We will walk through how to define the problem, engineer the features, and train, evaluate and deploy the machine learning model. 

## Why is Air Quality important?
Recent bush fire events in Australia is just one reminder of the importance of breathable air. According to the World Health Organization, air pollution is the 4th largest risk factor to human health worldwide, and 90% of the world breathes unhealthy air. Having realiable projections of air quality can help individuals as well as organizations to take steps to mitigate the health affects caused by dangerous air quality levels. For more information one non-profit's efforts to solve this global issue, visit [Open AQ](https://openaq.org/).

<figure>
<img src="img/syd_harb_air.jpg" width="600px" alt="Sydney Harbour Air During Bushfires"/>
<figcaption>Sydney Harbour during hazardous air quality conditions</figcaption>
</figure>

## What is time series analysis?
Time series analysis applies mathematical techniques to quantities that are ordered by time, in order to find insights about the past as well as the future. Historically, weather forecasting is one of the first time series analysis problem undertaken by humans. A early as neolithic times, civilizations used calendars as a means to predict seasonal patterns for agriculture. Time series problems exist in almost every domain. Predicting demand for future sales and services, forecasting utilization of compute resources and projecting call volume in call centers are all good examples of time series problems. Many of the methodologies used for time series analysis have been around for a long time. Long before deep learning techniques where even invented, algorithms like ARIMA have been in use since the 1950's. For many problems, these well developed techniques are still the best way to solve time series problems. The recent exponential growth in data and compute power has spawned the development of machine learning techniques based on neural networks that work well with larger and more complex data sets. One example of this type of data is air quality measurements, which are composed of millions of measurements from hundreds of locations. The DeepAR algorithm developed by Amazon research scientists to do time series forecasting on large data sets of related time series, makes training a forecasting model with this complex data possible.

>  **Definitions:** Univariate means a single value type. Multivariate means multiple value types. For example, a time series of temperature and humidity is multivariate, whereas a time series of temperature alone is univariate. Many time series algorithms only work with univariate date. Some algorithms work with multivariate data, but only predicts values for a single target value type. The other time series is called the related time series or the "exogenous" time series. The DeepAR algorithm that we will use in this example works with multivariate data, but we will only use univariate air quality data. To improve the quality of the predictive model, we could also use an exogenous time series, such as wind or temparature, but this out of scope for this project.

## Problem definition
The first step in any machine learning problem, is to understand the desired outcome.  You need to have a concrete goal to work towards through the entire process of discovery, design, development, deployment and operation. Try to answer these questions up front:

- Who will be consuming the predictions?
- How will the predictions be used? 
- How often do predictions need to be made?
- What are the minimum KPI's for the predictions in order for them to be useful?

Throughout your project, you should continously review that your work is meeting the end goals. Defining a clear problem statement up front is critical throughout the process. 

For this problem, the project sponsors have given a detailed use case description:
> *A state environmental protection agency wishes to provide air pollution estimates for particulate matter via a public web page. Predictions need to be made for over 100 locations throughout Australia. The forecast should be updated every hour, and needs to show the projected particulate matter 10 micron (pm10) values for the next 2 days on an hourly basis. A range of possible values should be shown. The generated static web page should have a link to a csv file of the predictions as well as a summary visualization. The predicted air quality should be averaged over 24 hours and classified according to the Victorian Air Quality standards for pm10. Healthy and unhealthy air days should be predicted correctly 75% of the time.*

<table>
    <tbody>
        <tr>
            <th>Air quality category</th>
            <th>
            <p><span>PM</span><sub><span>10</span></sub><span>&nbsp;µg/m</span><sup><span>3&nbsp;</span></sup><span>averaged over 1&nbsp;hour&nbsp;</span></p>
            </th>
        </tr>
        <tr>
            <th style="background-color: #64a13c; text-align: center;"><span style="color: white;">Good</span></th>
            <td>&nbsp; Less than 40</td>
        </tr>
        <tr>
            <th style="background-color: #eac51c; text-align: center;">Moderate</th>
            <td>&nbsp; 40–80</td>
        </tr>
        <tr>
            <th style="background-color: #d67900; text-align: center;"><span style="color: white;">Poor</span></th>
            <td>&nbsp; 80–120</td>
        </tr>
        <tr>
            <th style="background-color: #a90737; text-align: center;"><span style="color: white;">   Very poor</span></th>
            <td>&nbsp; 120–240</td>
        </tr>
        <tr>
            <th style="background-color: #50051e; text-align: center;"><span style="color: white;">Hazardous</span></th>
            <td>&nbsp; More than 240</td>
        </tr>
    </tbody>
</table>


Getting such a well defined problem statement is rarely easy. You will need to spend time with the project sponsors to understand what the desired end result is, and how this translates into machine learning requirements. Despite the difficulty in establishing thorough requirements, it's critical we figure this out up front to avoid developing a machine learning model that does not meet clearly defined objectives. 

> There is one thing that is not clear from the problem statement above. What is the mathematical way to express "healthy" versus "unhealthy"? After speaking with the project sponsors, we are told that any pm10 value > 80 is unhealthy.

From the problem statement there are several key points that determine what needs to be built:
- This is a 2 day forecast, with hourly frequency.
- The forecast will be created every hour using batch techniques.
- The probability distribution of predicted pm10 values is needed, not just a point forecast.
- We need to classify a 24 hour rolling average, according to the stated pm10 ranges for "good", "moderate", "poor", "very poor" and "hazardous".
- For evaluation, the percentage of time we correctly predict unhealthy versus healthy is calculated.
- There are no pre-existing benchmarks to beat. Sometimes when developing a new predictive model, it needs to beat an existing system.
- The application will generate a static html page for the visualizations and raw csv files for the predictions.


## General approach
Now that there is a defined problem, here are the steps needed:
1. Find a data source for pm10 values with at least one hour resolution for Australia.  
2. Explore the data and perform some analysis on its properties.
3. Perform feature engineering to transform the data into training and test data sets.
4. Train a machine learning model with the training data.
5. Infer predictions using the trained model on the test data.
6. Calculate the categorical class for the predicitons based on the ranges above.
7. Evaluate the trained model by comparing actuals versus predicted.
8. Create graphs and CSV files of the predictions.
9. Create a machine learning pipeline to automate all of the above.
> **Agility Is Important:** Machine learning projects are not linear. Many of the steps above could require repeating previous steps. For example, the data sources might be missing data, and the the problem statement needs to be reworked. Also, ehe evaluated model might not meet evaluation criteria, so additional data needs to be found. An agile approach that allows for experimentation, failure and redoing steps is required. 

## Data Discovery 
The data used to train the forecasting models needs to be found first. There are many open data sets available, as well as data from your own organizations. One good source of data is [Registry of Open Data on AWS](https://registry.opendata.aws). The registry has a simple search interface that can be used to find data. Lets search for "air quality" data sets:

![Registry of Open Data](img/rod_screen_shot.png)

The [OpenAQ](https://registry.opendata.aws/openaq/) data set has per city air quality measurements at hourly frequency, and is a perfect fit for the problem.

![Open AQ](img/openaq_screen_shot.png)

The data is publicy available on S3 and contains many locations and different quality measurements. It needs to be filtered down to only Australia pm10 measurements. By Amazon Athena to query the data, a smaller csv file containing only what is needed can be created.

### Import project dependencies
Before beggining, first import all the python modules, configuration settings and helper functions needed. 

> **Note:** When developing your own projects, it's recommended to separate lower level code out to a python module to make the notebook more readable. This also makes putting the final code into production format easier as it is already partly modularized.

In [1]:
from openaq.project_dependencies import *

### Create external Athena table
The table used to query against is backed by a publicly readable S3 bucket provided by Open AQ. The data definition language (DDL) query sets the external tables S3 location, and table definition, so it can be queried by Athena. A simple function that uses the boto3 API's to execute operations with Athena and wait for results is used to both create the table and query it. 

> Note: The table definition is based on this [code](https://gist.github.com/jflasher/573525aff9a5d8a966e5718272ceb25a). The code creates an external table in Amazon Athena. To save the OpenAQ organization on data transport costs out of region, run Athena queries in the US East region if possible.

In [2]:
create_results_uri = athena_create_table('openaq/sql/openaq.ddl')

Exception: query 10b7ca1e-66be-4ca2-90f4-69b0f932bcb7 failed with status FAILED

### Query data with Athena
The data manipulation language (DML) query queries the external OpenAQ table for the data required, and places it into an S3 bucket owned by this account. 

> **Try this:** If you would like to modify this lab to be more relavent to where you live, try copying and modifying the sql in the `openaq/sql` file to get data for your country. Not all countries have data. If you are interested in building your own air quality measurment sensor, do a search on "DIY air quality monitoring".

In [4]:
query_results_uri = athena_query_table('openaq/sql/sydney.dml')

> Exercise: Copy past the SQL statement above. Now go to the [Athena console](https://console.aws.amazon.com/athena/home), paste the SQL in the query window and modify it to get the average (use the "avg" function) value of the Carbon Monoxide (code "co") measurements for Melbourne.

## Feature engineering
Feature engineering is the process of transforming raw data into features that can be used for training and testing a machine learning model. Time series data can be stored in unsorted format, can have missing values or may be captured at a higher or lower frquency than what is needed. In addition, machine learning algorithms require the data be put in a standard format for both training and inference. The main steps taken to format the raw data are outlined here.

![Feature Engineering](img/feature_engineering.png)

### Step 1: Read raw data
The Athena query that was previously run places the query results in CSV format in an S3 bucket. The path to the csv object is a combination of a base path defined for all Athena query results and the query id. Using these values, we constuct an S3 path, and then read the data into an in memory panda data frame. Pandas is a data science library that makes it easy to process and explore time series data. Using the pandas csv function the query results can be read directly into an in memory data frame. 

> **Explore:** For more information on working with Pandas, check out the  [user guide](https://pandas.pydata.org/docs/user_guide/index.html).

In [5]:
print (f'reading {query_results_uri}')
raw = pd.read_csv(query_results_uri, parse_dates=['timestamp'])
raw.head()

reading s3://231935085477-openaq-lab/athena/results/5a1de731-ee24-4e79-be5b-8d399c530ad0.csv


,country,city,location,parameter,timestamp,value,point_latitude,point_longitude
0,AU,Sydney East,Randwick,pm10,2017-08-13 20:00:00,13.7,-33.933334,151.24194
1,AU,Sydney East,Rozelle,pm10,2017-08-13 20:00:00,8.6,-33.865833,151.16250
2,AU,Sydney East,Chullora,pm10,2017-08-13 20:00:00,8.4,-33.893890,151.04527
3,AU,Sydney East,Earlwood,pm10,2017-08-13 20:00:00,9.3,-33.917778,151.13472
4,AU,Sydney North-west,Richmond,pm10,2017-08-13 20:00:00,16.0,-33.618332,150.74583


### Step 2: Sort and index by location and time 
Before doing any transformations the raw data needs to be organized by time and locations. In pandas, we can easily sort by data frame columns, and then create an index for all the categorical columns as well as time. The names of index columns are refered to as "levels" in pandas. After executing the query below, you will notice that the readings are now sorted and grouped by the index levels.

In [6]:
categorical_levels = ['country', 'city', 'location', 'parameter']
index_levels = categorical_levels + ['timestamp']
indexed = raw.sort_values(index_levels, ascending=True)
indexed = indexed.set_index(index_levels)
indexed.head()

value  \
country city        location parameter timestamp                    
AU      Sydney East Chullora pm10      2016-04-09 20:00:00   12.2   
                                       2016-04-09 21:00:00   12.3   
                                       2016-04-09 22:00:00   12.4   
                                       2016-04-09 23:00:00   12.5   
                                       2016-04-10 00:00:00   12.8   

                                                            point_latitude  \
country city        location parameter timestamp                             
AU      Sydney East Chullora pm10      2016-04-09 20:00:00       -33.89389   
                                       2016-04-09 21:00:00       -33.89389   
                                       2016-04-09 22:00:00       -33.89389   
                                       2016-04-09 23:00:00       -33.89389   
                                       2016-04-10 00:00:00       -33.89389   

                                                            point_longitude  
country city        location parameter timestamp                             
AU      Sydney East Chullora pm10      2016-04-09 20:00:00        151.04527  
                                       2016-04-09 21:00:00        151.04527  
                                       2016-04-09 22:00:00        151.04527  
                                       2016-04-09 23:00:00        151.04527  
                                       2016-04-10 00:00:00        151.04527

> **Exercise:** Get the mean, minimum or maximum pm10 values for all of the selected data.

### Step 3: Downsample to hourly samples by maximum value
Downsampling combines multiple samples that may fall into the same window of time according to the sampling frequency. Some air quality instruments may measure values more than once and hour. Since we want to predict the peak value in any given hour, we will take the maximum of the values for each hour. Depending on the time series use case, you could also use the mean, first, last and minimum value for downsampling. 

> **Note**: Using max on lat/long columns is ok as long as all entries in a single time series have the same lat/long values, which is the case for this data.

In [7]:
downsampled = indexed.groupby(categorical_levels + [pd.Grouper(level='timestamp', freq='1H')]).max()
downsampled.head()

value  \
country city        location parameter timestamp                    
AU      Sydney East Chullora pm10      2016-04-09 20:00:00   12.2   
                                       2016-04-09 21:00:00   12.3   
                                       2016-04-09 22:00:00   12.4   
                                       2016-04-09 23:00:00   12.5   
                                       2016-04-10 00:00:00   12.8   

                                                            point_latitude  \
country city        location parameter timestamp                             
AU      Sydney East Chullora pm10      2016-04-09 20:00:00       -33.89389   
                                       2016-04-09 21:00:00       -33.89389   
                                       2016-04-09 22:00:00       -33.89389   
                                       2016-04-09 23:00:00       -33.89389   
                                       2016-04-10 00:00:00       -33.89389   

                                                            point_longitude  
country city        location parameter timestamp                             
AU      Sydney East Chullora pm10      2016-04-09 20:00:00        151.04527  
                                       2016-04-09 21:00:00        151.04527  
                                       2016-04-09 22:00:00        151.04527  
                                       2016-04-09 23:00:00        151.04527  
                                       2016-04-10 00:00:00        151.04527

> **Exercise:** Copy the expression above and modify it to downsample with the mean value.

### Step 4: Back fill missing values
A common problem with time series data is that values are often missing. It's a important to determine how many missing values there are in your data and perform filling. 

In order to fill missing values there are two steps. First we re-index the data for the desired frequency of 1 hour. This will create a NaN (not a number) entry for any missing values. Once we have the reindexed data we can then calculate some statistics on the number of NaN values. 

To check for missing values, we filter all non-null columns, group them by location, count them per group, and then descibe the summary stats.

In [8]:
def fill_missing_hours(df):
    df = df.reset_index(level=categorical_levels, drop=True)                                    
    index = pd.date_range(df.index.min(), df.index.max(), freq='1H')
    return df.reindex(pd.Index(index, name='timestamp'))

filled = downsampled.groupby(level=categorical_levels).apply(fill_missing_hours)
filled[filled['value'].isnull()].groupby('location').count().describe()

,value,point_latitude,point_longitude
count,18.0,18.0,18.0
mean,0.0,0.0,0.0
std,0.0,0.0,0.0
min,0.0,0.0,0.0
25%,0.0,0.0,0.0
50%,0.0,0.0,0.0
75%,0.0,0.0,0.0
max,0.0,0.0,0.0


For this data, are not may not be any missing values. Depending on the country the data was gathered from, there may be more. If there are missing values, the next step is to replace the NaN values with something else. For this case, we will linearly interpolate between the last know values for the pm10 measurement. By also rounding to two decimal places, all of the interpolated values will match the actual values precision. We will also fill any missing latitudes or longitudes with the first value available for each location.

In [9]:
filled['value'] = filled['value'].interpolate().round(2)
filled['point_latitude'] = filled['point_latitude'].fillna(method='pad')
filled['point_longitude'] = filled['point_longitude'].fillna(method='pad')

filled[filled['value'].isnull()].groupby('location').count().describe()

,value,point_latitude,point_longitude
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


### Step 5: Create features
Now that we have a contiguous data set at the proper frequency, we can transform the data frame into a format that is required for training and testing. The DeepAR algorithm we will be using to train the machine learning model, requires data in the following format. 

`{start: <start time of series>, target: [v1,v2,v3....], cat: [id1, id2, id3...]}`

For each location, the features are the start time for the time series, a contiguous list of all values, and an optional categorical array. This categorical array is composed of all the id's for country, city, location and measurment type.

> **Explore:** Read more about the [DeepAR feature format](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html#deepar-inputoutput).

#### Aggregate time series into rows
To get to the required format, a single row per time series needs to be created. This is done by aggregating each locations time series values into a single list per row, and by only retainng the timestamp of the first value.

In [10]:
aggregated = filled.reset_index(level=4)\
    .groupby(level=categorical_levels)\
    .agg(dict(timestamp='first', value=list, point_latitude='first', point_longitude='first'))\
    .rename(columns=dict(timestamp='start', value='target'))

In order to relate each time series to future predictions generated from the trained model, a unique id per time series is needed. Once there is a unique id, we can break the data into a metadata dataframe and a features dataframe.

In [11]:
aggregated['id'] = np.arange(len(aggregated))
aggregated.reset_index(inplace=True)
aggregated.set_index(['id']+categorical_levels, inplace=True)
aggregated.head()

,,,,,start,target,point_latitude,point_longitude
id,country,city,location,parameter,,,,
0,AU,Sydney East,Chullora,pm10,2016-04-09 20:00:00,"[12.2, 12.3, 12.4, 12.5, 12.8, 13.3, 13.3, 13....",-33.893890,151.04527
1,AU,Sydney East,Cook And Phillip,pm10,2019-09-08 21:00:00,"[5.2, 5.4, 5.6, 5.8, 6.0, 6.1, 6.4, 6.7, 7.2, ...",-33.893890,151.04527
2,AU,Sydney East,Earlwood,pm10,2016-04-09 20:00:00,"[11.2, 11.3, 11.5, 11.6, 11.4, 11.3, 11.2, 11....",-33.917778,151.13472
3,AU,Sydney East,Macquarie Park,pm10,2017-08-31 00:00:00,"[5.5, 5.3, 5.3, 5.3, 5.2, 4.9, 4.8, 4.8, 4.8, ...",-33.917778,151.13472
4,AU,Sydney East,Randwick,pm10,2016-04-09 20:00:00,"[20.2, 20.55, 20.9, 20.2, 19.1, 18.2, 17.8, 18...",-33.933334,151.24194


#### Transform lat/long to geometry
Since our time series also have a geographical location, we will also keep track of lat/long so we can use it for displaying our results on maps. GeoPandas enables geo searches within pandas data frames, and requires a geometry object which is created by combining the latitude and longitude columns into a single point geometry column. The code below does all of this.

In [12]:
metadata = gpd.GeoDataFrame(
    aggregated.drop(columns=['target','start']), 
    geometry=gpd.points_from_xy(aggregated.point_longitude, aggregated.point_latitude), 
    crs={"init":"EPSG:4326"}
)
metadata.drop(columns=['point_longitude', 'point_latitude'], inplace=True)

# missing lat/longs
#metadata.loc[pd.IndexSlice[:,'AU','South West Queensland','Miles Airport',:], 'geometry'] = Point(150.165, -26.809167) 
#metadata.loc[pd.IndexSlice[:,'AU','South West Queensland','Hopeland',:], 'geometry'] = Point(150.6655, -26.8841)

# set geometry index
metadata.set_geometry('geometry')

metadata.head()

,,,,,geometry
id,country,city,location,parameter,
0,AU,Sydney East,Chullora,pm10,POINT (151.04527 -33.89389)
1,AU,Sydney East,Cook And Phillip,pm10,POINT (151.04527 -33.89389)
2,AU,Sydney East,Earlwood,pm10,POINT (151.13472 -33.91778)
3,AU,Sydney East,Macquarie Park,pm10,POINT (151.13472 -33.91778)
4,AU,Sydney East,Randwick,pm10,POINT (151.24194 -33.93333)


#### Create a map plot
The map plot will display a map that contains all the locations for our air quality measurments. We are using the bokeh library to plot the map. We will use this map later to select which locations to view predictions for.

> **Explore:** For more information on plotting with bokeh, check out the [user guide](https://docs.bokeh.org/en/latest/index.html). Other popular plotting libraries include [matplotlib](https://matplotlib.org/) and [plotly](https://plotly.com/). 

In [13]:
output_notebook()
tile_provider = get_provider(CARTODBPOSITRON)
curdoc().theme = 'light_minimal'

map_df = metadata.to_crs("EPSG:3857").reset_index()
map_source = GeoJSONDataSource(geojson=map_df.to_json(na='drop'))

tooltips = [
    ('Country', '@country'),
    ('City', '@city'),
    ('Location', '@location')
]

map_plot = figure(
    title="Measurement Locations", 
    plot_width=800, plot_height=400, 
    x_axis_type="mercator", y_axis_type="mercator", 
    tooltips=tooltips, 
    tools="lasso_select"
)

map_plot.add_tile(tile_provider)
map_circles = map_plot.circle(x="x", y="y", size=5, fill_color="blue", fill_alpha=0.8, source=map_source)
show(map_plot)

Loading BokehJS ...

> **Exercise:** Try to change the plot size, color of the location, and add 'parameter' as a tooltip to the above plot.

#### Add categorical feature
The DeepAR model allows time series to be associated by categorical features. This is a key features, as we are building a single machine leanring model to predict air quality for many different locations. The categorical features enable the deep learning model to build an internal representation of how locations are related to each other. To do this, we create a list of ids that represent each of categorical features. The country, city, location, and measuremnt type will each be codified as an id, and combined into a list for each locations time series.

> **Note:** Since we only are building a model for pm10 values in Australia, only the city and location ids will have multiple values. We could modify the initial query to get multiple measurement types across many countries and train a more comprehensive model using exact the same steps we have walked through.

To get a set of categorical ids, we use the pandas factorize method to generate id's for categorical values. We then apply string to categorical conversion across every row of our aggregated data set.

In [14]:
level_ids = [level+'_id' for level in categorical_levels]
for l in level_ids:
    aggregated[l], index = pd.factorize(aggregated.index.get_level_values(l[:-3]))
    
aggregated['cat'] = aggregated.apply(lambda columns: [columns[l] for l in level_ids], axis=1)
features = aggregated.drop(columns=level_ids+ ['point_longitude', 'point_latitude'])
features.reset_index(level=categorical_levels, inplace=True, drop=True)
features.head()

,start,target,cat
id,,,
0,2016-04-09 20:00:00,"[12.2, 12.3, 12.4, 12.5, 12.8, 13.3, 13.3, 13....","[0, 0, 0, 0]"
1,2019-09-08 21:00:00,"[5.2, 5.4, 5.6, 5.8, 6.0, 6.1, 6.4, 6.7, 7.2, ...","[0, 0, 1, 0]"
2,2016-04-09 20:00:00,"[11.2, 11.3, 11.5, 11.6, 11.4, 11.3, 11.2, 11....","[0, 0, 2, 0]"
3,2017-08-31 00:00:00,"[5.5, 5.3, 5.3, 5.3, 5.2, 4.9, 4.8, 4.8, 4.8, ...","[0, 0, 3, 0]"
4,2016-04-09 20:00:00,"[20.2, 20.55, 20.9, 20.2, 19.1, 18.2, 17.8, 18...","[0, 0, 4, 0]"


### Step 6: Split features into training and a test sets
In order to evaluate the final model, the features need to be split into training and test sets. To accurately get an idea of how the model will perform on previously unseen data, time series data should be split according to a cutoff date.

![training and test split by time](img/train_test_split.png)

In [15]:
train_test_split_date = date.today() - timedelta(days=30)
train = filter_dates(features, None, train_test_split_date, '1H')
train.head()

,start,target,cat
id,,,
0,2016-04-09 20:00:00,"[12.2, 12.3, 12.4, 12.5, 12.8, 13.3, 13.3, 13....","[0, 0, 0, 0]"
1,2019-09-08 21:00:00,"[5.2, 5.4, 5.6, 5.8, 6.0, 6.1, 6.4, 6.7, 7.2, ...","[0, 0, 1, 0]"
2,2016-04-09 20:00:00,"[11.2, 11.3, 11.5, 11.6, 11.4, 11.3, 11.2, 11....","[0, 0, 2, 0]"
3,2017-08-31 00:00:00,"[5.5, 5.3, 5.3, 5.3, 5.2, 4.9, 4.8, 4.8, 4.8, ...","[0, 0, 3, 0]"
4,2016-04-09 20:00:00,"[20.2, 20.55, 20.9, 20.2, 19.1, 18.2, 17.8, 18...","[0, 0, 4, 0]"


In [16]:
test = filter_dates(features, train_test_split_date, None, '1H')
test.head()

,start,target,cat
id,,,
0,2020-07-20,"[22.64, 21.95, 21.27, 20.59, 19.91, 19.22, 18....","[0, 0, 0, 0]"
1,2020-07-20,"[7.12, 7.24, 7.35, 7.46, 7.57, 7.68, 7.79, 7.9...","[0, 0, 1, 0]"
2,2020-07-20,"[6.35, 6.63, 6.91, 7.18, 7.46, 7.74, 8.01, 8.2...","[0, 0, 2, 0]"
3,2020-07-20,"[6.96, 7.09, 7.22, 7.35, 7.48, 7.61, 7.74, 7.8...","[0, 0, 3, 0]"
4,2020-07-20,"[12.54, 12.45, 12.37, 12.29, 12.21, 12.12, 12....","[0, 0, 4, 0]"


## Train DeepAR model
Now that we have our features split into to train and test sets, we can train a machine learnign model. For this problem, we will use DeepAR, an first party algorythm available in SageMaker. DeepAR is a deep leanring algorithm that creates a single machine learning model for multiple related time series.

### Create estimator
To create an estimator, get the image name for the desired algorithm, the execution role and determin the number of training instances and type needed. For DeepAR a gpu accelerated instance is needed.

In [18]:
session = sagemaker.Session()
region = session.boto_region_name
image_uri = sagemaker.amazon.amazon_estimator.get_image_uri(region, 'forecasting-deepar', 'latest')
output_path = f's3://{bucket_name}/sagemaker/output'

training_job_name=None
if training_job_name:
    estimator = sagemaker.estimator.Estimator.attach(
        training_job_name, 
        sagemaker_session= session
    )
    
else:
    estimator = sagemaker.estimator.Estimator(
        sagemaker_session= session,
        image_name= image_uri,
        role= sagemaker.get_execution_role() ,
        train_instance_count= 1,
        train_instance_type='ml.p2.xlarge',
        base_job_name= 'deepar-openaq-demo',
        output_path= output_path
    )

### Upload training data to S3
In order to use the training data, we need to get it from in memory pandas data frames to a location in S3 where SageMaker can use it. We first write all of the train and test set features to a local location. We can then use the SageMaker upload_data method to upload them to S3, and get a reference id that is used later by SageMaker.

In [20]:
train.to_json('openaq/data/train.json', orient='records', lines=True)
test.to_json('openaq/data/test.json', orient='records', lines=True) 

data = dict(
    train= session.upload_data(path='openaq/data/train.json'),
    test=  session.upload_data(path='openaq/data/test.json')
)

### Create a hyperparameter optimization (HPO) job
Machine learning models have tunable parameters that need to be set prior to training a model. These parameters affect both the time it takes to train a model and the quality metrics of the trained model. Hyperparameter optimization jobs train multiple models across ranges of these parameters to find the best combinations for the given data set. SageMaker HPO uses Bayesian optimization to rapidly find the optimal parameters much faster then a random or grid search. There are two sets of parameters below. The parameters with a range of values will be optimized, and static parameters won't be. For more information on the DeepAR hyper-parameters, visit the [developer guide.](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html)

![hyper-parameter tuning](img/hpo.png)

> Note: HPO does not need to run during every training cycle. Because it is training multiple models, it is time consuming and expensive. Typically, you run HPO once, and then use static hyperparameters until the the ML models are no longer meeting evaluation metric goals. Since I have already run a large HPO job to determine the hyperparameters, they are set statically below, and we will only start the HPO job to demostrate how it operates. If you are training with a different country or measurement type, re-running a full HPO job will give you improved results.

#### Set static hyperparameters

The static parameters are the ones we know to be the best based on previously run HPO jobs, as well as the non-tunable parameters like prediction length and time frequency that are set according to requirements.

In [22]:
hpo = dict(
    time_freq= '1H'
    ,early_stopping_patience= 40
    ,prediction_length= 48
    ,num_eval_samples= 10
    ,test_quantiles= [0.5, 0.7, 0.9]
    
    # not setting these since HPO will use range of values
    #,epochs= 400
    #,context_length= 3
    #,num_cells= 157
    #,num_layers= 4
    #,dropout_rate= 0.04
    #,embedding_dimension= 12
    #,mini_batch_size= 633
    #,learning_rate= 0.0005
)

#### Set hyper-parameter ranges
The hyperparameter ranges define the parameters we want the tuner to search across.

> Explore: Look in the [user guide](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html) for DeepAR and add the recommended ranges for `embedding_dimension` to the below.

In [23]:
hpo_ranges = dict(
    epochs= IntegerParameter(1, 1000)
    ,context_length= IntegerParameter(7, 48)
    ,num_cells= IntegerParameter(30,200)
    ,num_layers= IntegerParameter(1,8)
    ,dropout_rate= ContinuousParameter(0.0, 0.2)
    ,embedding_dimension= IntegerParameter(1, 50)
    ,mini_batch_size= IntegerParameter(32, 1028)
    ,learning_rate= ContinuousParameter(.00001, .1)
)

#### Create and start the HPO job
Once we have the HPO tuner defined, the fit method is called. This will trigger the launching of the training instances. When creating the tuner, an objective metrice is needed to compare each model it trains. For this model, the tuner will find the model with the minimal final loss.

In [24]:
estimator.set_hyperparameters(**hpo)

hpo_tuner = HyperparameterTuner(
    estimator= estimator, 
    objective_metric_name= 'train:final_loss',
    objective_type='Minimize',
    hyperparameter_ranges= hpo_ranges,
    max_jobs=2,
    max_parallel_jobs=1
)

hpo_tuner.fit(data)
hpo_job_name = hpo_tuner.latest_tuning_job.name

display(HTML(f'''<br>The hyperparameter tuning job "{hpo_job_name}" is now running. 
        To view it in the console click 
        <a href="https://console.aws.amazon.com/sagemaker/home?region={region}#/hyper-tuning-jobs">here</a>.
    '''))    

INFO:sagemaker.tuner:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: forecasting-deepar-200819-1304


The tuning job will run for a few hours, as it will be traing several machine learning models, each of which can take as long as 2 hours. Instead of waiting, we will use a model that created from a previous HPO job that ran for two days, an compared 150 models. The code below will stop the running HPO job, and then copy this model artifact to a location in S3 to simulate the final output of an HPO job. 

> **Note:** If you are running this lab and want to optimize on a different set of data be sure to comment out the code below to allow your hpo job to complete, and give you a customized model. Warning! This will take some time to complete!

In [26]:
hpo_tuner.stop_tuning_job()
hpo_tuner = None

AttributeError: 'NoneType' object has no attribute 'stop_tuning_job'

In order to wait for the HPO job to complete, call wait on the tuner. The HPO job will take many hours to complete if it was not stopped above.

In [27]:
if hpo_tuner: 
    hpo_tuner.wait()

### Train model on complete data set
The HPO job we ran above was not trained on the most recent data. Since we now have the best hyper-parameters, we will retrain a model on the most up to date data so any current patterns in air quality will be learned by the model. In production, we would normally skip over the hyperparameter optimization job, and just retrain our model on a daily basis prior to creating inferences. If the qaulity of our forecasts in production starts to go down, or we are aware of big changes to the data sources, for example if new countries are added, we would rerun the hpo job to get new hyper-parameters.

![model training](img/training.png)

In [ ]:
# use full set of data for training
all_data.to_json('data/all_data.json', orient='records', lines=True)
data = dict(train= session.upload_data(path='data/all_data.json'))

hpo['num_layers'] = 4 # set previous hpo range to a static value
estimator.set_hyperparameters(**hpo)
estimator.fit(data)

### Deploy best model as an endpoint


In [ ]:
if endpoint_name:
    predictor = RealTimePredictor(endpoint_name)
    
elif training_job_name:
    predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c4.2xlarge')

else:
    hpo_tuner.wait()
    predictor = hpo_tuner.deploy(initial_instance_count=1, instance_type='ml.c4.2xlarge')

## Create inferences (predictions)
### Generate test sets to predict

In [ ]:
test_dates = pd.date_range(test_start_date, periods=test_periods, freq=f'{frequency}H')
tests = get_tests(test, test_dates, frequency, context_length, prediction_length)
tests

### Call the endpoint

In [ ]:
generate_predictions = False
if generate_predictions:
    quantile_strs = [f'0.{q}' for q in quantiles]
    predictions = predict(endpoint_name, tests, quantile_strs)
    predictions.to_pickle(predictions_file)
else:
    predictions = pd.read_pickle(predictions_file)

predictions 

## Graph the results

#### Create predictions data source

In [ ]:
create_indexdb_tables()
index_prediction_data()

#### Create prediction plot
Now we will create a plot with the actual values and the quatiles from the predictions. The initial data sources will be empty until we select a location.


In [ ]:
actuals = filter_dates(features, graph_start_date, graph_end_date, frequency).drop(columns=['cat'])
actuals_source = ColumnDataSource(dict(id=[], start=[], target=[])) # empty

filtered_predictions = predictions.reset_index(level=1)
predictions_source = ColumnDataSource(dict(id=[], start=[], **{f'0.{q}=[] for q in quantiles}))
                                                               
# create the plot
predictions_plot = figure(
    title='', 
    plot_width=800, plot_height=400, 
    x_axis_label='date/time', 
    y_axis_label='pm10 ',
    x_axis_type='datetime',
    y_range= [0, max(predictions['0.5'].max())],
    tools=''
)

# plot vertical areas for the quantiles
quantile_names = [f'0.{q}' for q in quantiles]
predictions_plot.varea_stack(
    stackers=quantile_names, 
    x='start', 
    color= inferno(len(quantiles)), 
    legend_label=quantile_names, 
    source=predictions_source,
    alpha=1,
)

# plot actual values
predictions_plot.line(
    x= "start", y= "target", 
    color= 'white', 
    source= actuals_source
)

# add a legend
predictions_plot.legend.items.reverse()

#### Create location selector

In [ ]:
location_select = Select(title='Select Location:', value=' ', options=[])

#### Create prediction start slider

In [ ]:
start_min = filtered_predictions.reset_index()['start'].min()
start_slider = Slider(
    start=0, 
    end=test_periods-1, 
    value=0, 
    step=1, 
    title=f'prediction time delta'
)

#### Create javascript callback
The javascript callback function connects all the plots and gui components together so changes will update the plots.

In [ ]:
callback_args=dict(
    map_source= map_source, 
    actuals= actuals_source, 
    predictions= predictions_source, 
    location_select= location_select, 
    start_slider= start_slider
)

with open('javascript/map_update_callback.js', 'r') as f:
    callback_code = f.read()
    map_update_callback = CustomJS(code=callback_code, args=callback_args)
    map_source.selected.js_on_change('indices', map_update_callback)

with open('javascript/plot_update_callback.js', 'r') as f:
    callback_code = f.read()    
    plot_update_callback = CustomJS(code=callback_code, args=callback_args)
    location_select.js_on_change('value', plot_update_callback)
    start_slider.js_on_change('value', plot_update_callback)

#### Show the gui
Once we have created all the elements, we can now show the entire air quality GUI.

In [ ]:
show(column(map_plot, location_select, predictions_plot, start_slider))

## Generate Keplar.gl map

In [ ]:
#from keplergl import KeplerGl 
#map_1 = KeplerGl(height=500)
#map_1.add_data(data=predictions, name='sydney_2020')
#map_1.save_to_html()

## Image Attributions
<p style="font-size: 0.9rem;font-style: italic;"><img style="display: block;" src="https://live.staticflickr.com/65535/49246056803_0a0bae48cf_b.jpg" border="1px" solid="#ddd" border-radius="4px" padding="5px" width="150px" alt="P1210668"><a href="https://www.flickr.com/photos/37912374670@N01/49246056803">"P1210668"</a><span> by <a href="https://www.flickr.com/photos/37912374670@N01">acb</a></span> is licensed under <a href="https://creativecommons.org/licenses/by-nc-sa/2.0/?ref=ccsearch&atype=html" style="margin-right: 5px;">CC BY-NC-SA 2.0</a><a href="https://creativecommons.org/licenses/by-nc-sa/2.0/?ref=ccsearch&atype=html" target="_blank" rel="noopener noreferrer" style="display: inline-block;white-space: none;margin-top: 2px;margin-left: 3px;height: 22px !important;"><img width="15px"style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc_icon.svg" /><img width="15px" style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc-by_icon.svg" /><img width="15px" style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc-nc_icon.svg" /><img width="15px" style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc-sa_icon.svg" /></a></p>